In [7]:
# write code that loads in all "summary.csv" files in all subdirectories of a given directory and concatenates them into a single dataframe
import os
import pandas as pd

def load_summary_files(directory):
    dfs = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == "summary.csv":
                df = pd.read_csv(os.path.join(root, file))
                dfs.append(df)
    return pd.concat(dfs)

results_full = load_summary_files("webagents-step-main/data/webarena/eval/step_full/gpt-4-turbo-2024-04-09/run1")
results_reddit = load_summary_files("webagents-step-main/data/webarena/eval/step_reddit_unmodified/gpt-4-turbo-2024-04-09/run1")

In [9]:
# check which tasks are missing
reddit_ids = [27, 28, 29, 30, 31, 66, 67, 68, 69, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 552, 553, 554, 555, 562, 563, 564, 565, 566, 580, 581, 582, 583, 584, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 671, 672, 673, 674, 675, 681, 682, 683, 684, 685, 686, 687, 688, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 791]
missing_tasks = set(reddit_ids) - set(results_reddit["task_id"])
print(missing_tasks)

# one task could not be completed on reddit by the agent that raised error. We are counting this is failure whe computing the success rate below

{598}


In [8]:
# check that all tasks have been completed
print(results_full.task_id.nunique())
print(results_reddit.task_id.nunique()) # one task could not be completed and agent raised error


812
128


In [12]:

(results_full.success.sum())/812
(results_reddit.success.sum())/129

print("Success Rate STeP full benchmark: ", results_full.success.sum()/812)
print("Success Rate STeP Reddit benchmark: ", results_reddit.success.sum()/129)

Success Rate STeP full benchmark:  0.24753694581280788
Success Rate STeP Reddit benchmark:  0.21705426356589147


## the below code block generates a dict with all task_ids per website that have that website as start url

In [5]:
# for all .json files in a directoy, load the file and extract the "task_id". append the task_ids to a dict that is populated with the keys being the first item in the list "sites" that is part of each json file
import json
import os

task_ids = {}
for root, dirs, files in os.walk("/scratch/gpfs/bs6865/agent-eval/webagents-step-main/tasks/webarena"):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(root, file)) as f:
                data = json.load(f)
                site = data["sites"][0]
                if site == "wikipedia":
                    site = data["sites"][1]
                task_id = data["task_id"]
                if site not in task_ids:
                    task_ids[site] = []
                task_ids[site].append(task_id)